In [7]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from scipy import stats

def statistici(file_path, sheet1_name, sheet2_name):
   # Citirea tabelului de emisii
    df_emissions = pd.read_excel(file_path, sheet_name=sheet1_name, index_col=0)
    df_emissions.index = df_emissions.index.str.strip()  # Eliminarea spațiilor suplimentare din numele țării

    # Citirea tabelului de cheltuieli
    df_expenditure = pd.read_excel(file_path, sheet_name=sheet2_name, index_col=0)
    df_expenditure.index = df_expenditure.index.str.strip()  # Eliminarea spațiilor suplimentare din numele țării


    # Înlocuim valorile ":" cu numpy.nan în tabelul de cheltuieli
    df_expenditure.replace(":", np.nan, inplace=True)
    df_emissions.replace(":", np.nan, inplace=True)
    

    # Gestionăm valorile lipsă în tabelul de cheltuieli, înlocuind NaN cu cea mai apropiată valoare din coloana respectivă (backward fill)
    df_expenditure = df_expenditure.fillna(method='bfill')
    df_emissions = df_emissions.fillna(method='bfill')
    

    # Conversia valorilor în float, astfel încât să putem calcula corelația
    df_expenditure = df_expenditure.astype(float)
    df_emissions = df_emissions.astype(float)
    

    # Identificăm corelațiile semnificative și coeficienții de determinare pentru fiecare țară
    significant_correlations = {}

    for country in df_emissions.index:
        X = df_expenditure.loc[country].values
        y = df_emissions.loc[country].values
        # Verificăm dacă cele două variabile sunt constante (toate valorile sunt aceleași)
        if np.all(X == X[0]) or np.all(y == y[0]):
            r_squared = 0  # Setăm coeficientul de determinare la 0 în acest caz
            p_value = 1    # Setăm p-value la 1 (nu este semnificativ)
        else:
            model = LinearRegression()
            model.fit(X.reshape(-1, 1), y)
            r_squared = model.score(X.reshape(-1, 1), y)
            _, p_value = stats.pearsonr(X, y)

        significant_correlations[country] = {"r_squared": r_squared, "p_value": p_value}


    # Sortăm țările în funcție de coeficientul de determinare (R^2) în ordine descrescătoare
    sorted_countries = sorted(significant_correlations.items(), key=lambda x: x[1]['p_value'])

    # Identificăm primele trei țări cu cele mai mari coeficienți de determinare și p-valori mici
    top_3_countries = []
    for country, values in sorted_countries:
        if values['r_squared'] > 0.4 and values['p_value'] < 0.05:
            top_3_countries.append((country, values))
            if len(top_3_countries) == 3:
                break

    # Identificăm ultimele trei țări cu cele mai mici coeficienți de determinare și p-valori mari
    bottom_3_countries = []
    for country, values in reversed(sorted_countries):
        bottom_3_countries.append((country, values))
        if len(bottom_3_countries) == 3:
            break

    # Identificăm poziția României în clasament
    rank_of_romania = next((i + 1 for i, (country, _) in enumerate(sorted_countries) if country == 'Romania'), None)

    # Calculam numarul de tari pentru a afisa clasamentul
    num_of_countries = df_emissions.shape[0]

    print("Primele trei țări cu cele mai mari coeficienți de determinare și corelatie mare:")
    for i, (country, values) in enumerate(top_3_countries):
        print(f"{i + 1}. {country}: Corelatie: {1 - values['p_value']:.4f}, Coeficient de determinare (R^2): {values['r_squared']:.2f}")

    print("\nUltimele trei țări cu cele mai mici coeficienți de determinare și corelatie mica:")
    for i, (country, values) in enumerate(reversed(bottom_3_countries)):
        print(f"{num_of_countries - 2 + i}. {country}: Corelatie: {1 - values['p_value']:.4f}, Coeficient de determinare (R^2): {values['r_squared']:.2f}")

    # Afișăm poziția României în clasament
    print(f"\nRomânia se află pe poziția {rank_of_romania} în clasament.")


### Corelatie Emisii/Linii electrice

In [8]:
# Încarcăm fișierul Excel
file_path = r'C:\Users\Ricardo Milos\Documents\practica\Toate periate.xlsx'  # Schimbă calea cu locația reală a fișierului
sheet1_name = 'electric locomotives'
sheet2_name = 'Net greenhouse gas emissions 19'

#Am adus cateva modificari codului si l-am rulat pentru tabelele electric locomotives si greenhouse gas emissions

statistici (file_path, sheet1_name, sheet2_name)

Primele trei țări cu cele mai mari coeficienți de determinare și corelatie mare:
1. Poland: Corelatie: 0.9993, Coeficient de determinare (R^2): 0.87
2. Czechia: Corelatie: 0.9989, Coeficient de determinare (R^2): 0.85
3. Belgium: Corelatie: 0.9563, Coeficient de determinare (R^2): 0.52

Ultimele trei țări cu cele mai mici coeficienți de determinare și corelatie mica:
23. Lithuania: Corelatie: 0.0000, Coeficient de determinare (R^2): 0.00
24. Luxembourg: Corelatie: 0.0000, Coeficient de determinare (R^2): 0.00
25. Slovenia: Corelatie: 0.0000, Coeficient de determinare (R^2): 0.00

România se află pe poziția 20 în clasament.


### Tabele r, r^2, p-value

In [15]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from scipy import stats

def calculate_correlation_statistics(file_path, sheet1_name, sheet2_name):
    # Citirea datelor din fișierul Excel
    df_emissions = pd.read_excel(file_path, sheet_name=sheet1_name, index_col=0)
    df_expenditure = pd.read_excel(file_path, sheet_name=sheet2_name, index_col=0)

    # Înlocuirea valorilor ":" cu NaN în tabelul de cheltuieli
    df_expenditure.replace(":", np.nan, inplace=True)
    df_emissions.replace(":", np.nan, inplace=True)

    # Gestionarea valorilor lipsă în tabelul de cheltuieli prin completarea cu cea mai apropiată valoare
    df_expenditure = df_expenditure.fillna(method='bfill')
    df_emissions = df_emissions.fillna(method='bfill')

    # Conversia valorilor în float
    df_expenditure = df_expenditure.astype(float)
    df_emissions = df_emissions.astype(float)

    # Crearea unui nou DataFrame pentru rezultate
    result_df = pd.DataFrame(columns=['r', 'r^2', 'p-value'])

    # Parcurgerea țărilor care se găsesc în ambele tabele
    common_countries = df_emissions.index.intersection(df_expenditure.index)
    for country in common_countries:
        X = df_expenditure.loc[country].values
        y = df_emissions.loc[country].values

        # Verificăm dacă cele două variabile sunt constante (toate valorile sunt aceleași)
        if np.all(X == X[0]) or np.all(y == y[0]):
            r = 0         # Setăm coeficientul de corelație la 0 în acest caz
            r_squared = 0  # Setăm coeficientul de determinare la 0 în acest caz
            p_value = 1    # Setăm p-value la 1 (nu este semnificativ)
        else:
            r, _ = stats.pearsonr(X, y)
            r_squared = r**2
            _, p_value = stats.pearsonr(X, y)

        result_df.loc[country] = [r, r_squared, p_value]

    return result_df

# Încărcarea fișierului Excel și specificarea numelor celor două foi de lucru
file_path = r'C:\Users\Ricardo Milos\Documents\practica\Toate periate.xlsx'  # Schimbă calea cu locația reală a fișierului
sheet1_name = 'electric locomotives'
sheet2_name = 'Net greenhouse gas emissions 19'

# Calcularea statisticilor de corelație și crearea DataFrame-ului cu rezultate
correlation_stats_df = calculate_correlation_statistics(file_path, sheet1_name, sheet2_name)


# Afișarea rezultatelor
print(correlation_stats_df)



                    r       r^2       p-value
Austria      0.977057  0.954641  2.967431e-05
Belgium      0.972351  0.945467  5.175135e-05
Bulgaria     0.972351  0.945467  5.175135e-05
Croatia      0.908839  0.825989  1.766790e-03
Czechia      0.147891  0.021872  7.267213e-01
Denmark      0.197435  0.038981  6.393166e-01
Estonia      0.000000  0.000000  1.000000e+00
Finland      0.999894  0.999788  2.998674e-12
France      -0.268023  0.071836  5.210053e-01
Greece       0.789927  0.623984  1.967860e-02
Hungary     -0.476206  0.226772  2.329177e-01
Ireland     -0.817239  0.667880  1.324580e-02
Italy       -0.817239  0.667880  1.324580e-02
Latvia       0.000000  0.000000  1.000000e+00
Lithuania    0.000000  0.000000  1.000000e+00
Luxembourg   0.000000  0.000000  1.000000e+00
Netherlands  0.788257  0.621349  2.012441e-02
Poland       0.788257  0.621349  2.012441e-02
Portugal     0.998075  0.996153  1.781465e-08
Romania      0.936789  0.877574  6.018561e-04
Slovakia     0.843640  0.711729  8